In [ ]:
import pandas as pd
import xmltodict
import numpy as np
from IPython.display import display, HTML
from tqdm import tqdm


def runProcess():
    print('--> Computing data ...')
    tableFrame = []
    dict_results = {}
    dict_results[''] = ['Actives Entities',
                        'Total Synonyms in Actives Entities',
                        'Actives Entities with a Synonym',
                        '% of Actives Entities with a Synonym',
                        'Median number of synonyms in Actives Entities',
                        'Maximum number of synonyms in Actives Entities',
                        'Nomenclature Terms in Actives Entities',
                        'Groups of Disorders',
                        'Total Synonyms in Groups of Disorders',
                        'Groups of Disorders with a Synonym',
                        '% of Groups of Disorders with a Synonym',
                        'Median number of synonyms in Groups of Disorders ',
                        'Maximum number of synonyms in Groups of Disorders ',
                        'Nomenclature Terms in Groups of Disorders',
                        'Disorders',
                        'Total Synonyms in Disorders',
                        'Disorders with a Synonym',
                        '% of Disorders with a Synonym',
                        'Median number of synonyms in Disorders',
                        'Maximum number of synonyms in Disorders',
                        'Nomenclature Terms in Disorders',
                        'Subtypes of Disorders',
                        'Total Synonyms in Subtypes of Disorders',
                        'Subtypes of Disorders with a Synonym',
                        '% of Subtypes of Disorders with a Synonym',
                        'Median number of synonyms in Subtypes of Disorders',
                        'Maximum number of synonyms in Subtypes of Disorders',
                        'Nomenclature Terms in Subtypes of Disorders'
                        ]

    dict_language = {'en': 'English',
                     'cs': 'Czech',
                     'de': 'German',
                     'es': 'Spanish',
                     'fr': 'French',
                     'it': 'Italian',
                     'nl': 'Dutch',
                     'pl': 'Polish',
                     'pt': 'Portuguese'
                     }
    for iso, language in tqdm(dict_language.items()):
        dict_results[language] = []
        xml_data = getData('../xml/' + iso + '_product1.xml')

        ## ACTIVES
        actives_entities = getActivesEntities(xml_data)
        n_actives_entities = len(actives_entities)
        dict_results[language].append(int(n_actives_entities))
        n_entitiesWithSynonyms, total_number_of_synonyms, medianSyns, MaxSyns = getSynonymsStatistics(actives_entities)
        percent_n_ActivesWithSynonyms = str(round((n_entitiesWithSynonyms / n_actives_entities) * 100, 2))
        n_nomenclatureTerms = getNomenclatureTerms(n_actives_entities, total_number_of_synonyms)
        dict_results[language].append(int(total_number_of_synonyms))
        dict_results[language].append(int(n_entitiesWithSynonyms))
        dict_results[language].append(percent_n_ActivesWithSynonyms)
        dict_results[language].append(int(medianSyns))
        dict_results[language].append(int(MaxSyns))
        dict_results[language].append(int(n_nomenclatureTerms))

        ## GROUPS OF DISORDERS
        groupsOfDisorders = getGroupsOfDisorders(actives_entities)
        n_groupsOfDisorders = len(groupsOfDisorders)
        dict_results[language].append(n_groupsOfDisorders)
        n_groupsWithSynonyms, total_number_of_synonyms, medianSyns, MaxSyns = getSynonymsStatistics(groupsOfDisorders)
        percent_n_groupsWithSynonyms = str(round((n_groupsWithSynonyms / n_groupsOfDisorders) * 100, 2))
        n_nomenclatureTerms = getNomenclatureTerms(n_groupsOfDisorders, total_number_of_synonyms)
        dict_results[language].append(total_number_of_synonyms)
        dict_results[language].append(n_groupsWithSynonyms)
        dict_results[language].append(percent_n_groupsWithSynonyms)
        dict_results[language].append(medianSyns)
        dict_results[language].append(MaxSyns)
        dict_results[language].append(n_nomenclatureTerms)

        ## DISORDERS
        disorders = getDisorders(actives_entities)
        n_disorders = len(disorders)
        dict_results[language].append(n_disorders)
        n_disordersWithSynonyms, total_number_of_synonyms, medianSyns, MaxSyns = getSynonymsStatistics(disorders)
        percent_n_disordersWithSynonyms = str(round((n_disordersWithSynonyms / n_disorders) * 100, 2))
        n_nomenclatureTerms = getNomenclatureTerms(n_disorders, total_number_of_synonyms)
        dict_results[language].append(total_number_of_synonyms)
        dict_results[language].append(n_disordersWithSynonyms)
        dict_results[language].append(percent_n_disordersWithSynonyms)
        dict_results[language].append(medianSyns)
        dict_results[language].append(MaxSyns)
        dict_results[language].append(n_nomenclatureTerms)

        ## SUBTYPES OF DISORDERS
        subtypesOfDisorders = getSubtypesOfDisorders(actives_entities)
        n_subtypesOfDisorders = len(subtypesOfDisorders)
        dict_results[language].append(n_subtypesOfDisorders)
        n_subtypesWithSynonyms, total_number_of_synonyms, medianSyns, MaxSyns = getSynonymsStatistics(
            subtypesOfDisorders)
        percent_n_subtypesWithSynonyms = str(round((n_subtypesWithSynonyms / n_subtypesOfDisorders) * 100, 2))
        n_nomenclatureTerms = getNomenclatureTerms(n_subtypesOfDisorders, total_number_of_synonyms)
        dict_results[language].append(total_number_of_synonyms)
        dict_results[language].append(n_subtypesWithSynonyms)
        dict_results[language].append(percent_n_subtypesWithSynonyms)
        dict_results[language].append(medianSyns)
        dict_results[language].append(MaxSyns)
        dict_results[language].append(n_nomenclatureTerms)
    myDataframe = pd.DataFrame(dict_results)
    with pd.ExcelWriter('../output_tables/NumberOfSynonymsAndNomenclatureTerms.xlsx', engine='xlsxwriter') as writer:
        myDataframe.to_excel(writer,index=False)
    display(HTML(myDataframe.to_html(index=False)))

def getData(xmlfile):
    """
    Read an xml return a dict with xmltodict package
    :return: xml parsed as dict
    """
    with open(xmlfile, "r", encoding='ISO-8859-1') as ini:
        xml_dict = xmltodict.parse(ini.read())
    return xml_dict

def getActivesEntities(xml_data):
    """
    filter out inactive entities highlighting flag 8192
    :param xml_data:
    :return:
    """
    actives_entities = []
    for entity in xml_data["JDBOR"]["DisorderList"]["Disorder"]:
        DisorderFlagList = entity['DisorderFlagList']['DisorderFlag']
        if isinstance(DisorderFlagList, dict):
            if DisorderFlagList['Value'] != '8192':
                actives_entities.append(entity)
        else:
            DisorderFlags = []
            for DisorderFlag in DisorderFlagList:
                DisorderFlags.append(DisorderFlag['Value'])
            if not '8192' in DisorderFlags:
                actives_entities.append(entity)
    return actives_entities

def getGroupsOfDisorders(actives_entities):
    """
    keeping only groups of disorders (with flag 36547)
    :param actives_entities:
    :return:
    """
    disorders = []
    for entity in actives_entities:
        if entity['DisorderGroup']['@id'] == '36540':
            disorders.append(entity)
    return disorders

def getDisorders(actives_entities):
    """
    keeping only disorders (with flag 36547)
    :param actives_entities:
    :return:
    """
    disorders = []
    for entity in actives_entities:
        if entity['DisorderGroup']['@id'] == '36547':
            disorders.append(entity)
    return disorders

def getSubtypesOfDisorders(actives_entities):
    """
    keeping only subtypes of disorders (with flag 36547)
    :param actives_entities:
    :return:
    """
    disorders = []
    for entity in actives_entities:
        if entity['DisorderGroup']['@id'] == '36554':
            disorders.append(entity)
    return disorders

def getSynonymsStatistics(entities):
    """
    Generic method to get statistics on synonyms
    :param entities:
    :return:
    """
    list_n_synonyms = []
    for entity in entities:
        if entity['SynonymList']['@count'] != '0':
            list_n_synonyms.append(int(entity['SynonymList']['@count']))
    synonym_array = np.array(list_n_synonyms)
    total_number_of_synonyms = 0
    for n_synonym in list_n_synonyms:
        total_number_of_synonyms += n_synonym
    return len(list_n_synonyms) , total_number_of_synonyms, int(np.median(synonym_array)),np.max(synonym_array)

def getNomenclatureTerms(n_entities, n_synonyms):
    """
    Generic method to get statistics on nomenclature terms
    :param entities:
    :return:
    """
    n_nomenclatureTerms = n_entities + n_synonyms
    return n_nomenclatureTerms



if __name__ == '__main__':

    runProcess()
